# Activity 3 : Généralisation à plusieurs énigmes avec Selenium

<div style="text-align: center;">
    <img src="../images/scrappeur.png" width="600" height="300">
</div>

___

Pour cette activité, nous allons créer un tableau plus complet et pour plusieurs énigmes.

Cette fois-ci, on veut un tableau qui contiendra les colonnes suivantes : `title`, `num_enigme`, `url`, `image`, `enigme`, `solution` pour 5 énigmes.

Nous allons fournir la première partie pour sélectionner 5 énigmes. On commence par aller sur une page qui contient les liens vers toutes les énigmes du jeu : 

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random
import pandas as pd

In [2]:
url = "https://professeur-layton.fandom.com/fr/wiki/Cat%C3%A9gorie:%C3%89nigmes"

On se connecte en mode "headless" pour gagner quelques lignes de code :

In [3]:
# Création d'une instance des options Chrome
chrome_options = Options()

# Définition des options Chrome 
chrome_options.add_argument('--disable-search-engine-choice-screen') 
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--headless=new')
# Création d'une nouvelle instance du navigateur Chrome avec les options spécifiées
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

On récupère tous les liens de type href vers les énigmes grâce à un sélecteur type CSS_Selector : 

In [15]:
# Récupérer tous les éléments <a> avec la classe "category-page__member-link"
elements = driver.find_elements(By.CSS_SELECTOR, "a.category-page__member-link")
# Extraire les hrefs des éléments trouvés
hrefs = [element.get_attribute("href") for element in elements]

In [24]:
# Extraire les hrefs des éléments trouvés et filtrer pour exclure ceux contenant "Categories"
hrefs = [element.get_attribute("href") for element in elements if "Cat%C3%A9gorie" not in element.get_attribute("href")]

On extrait aléatoirement une liste de 3 liens (pour éviter d'avoir un code trop long et de surcharger le wiki en requêtes) :

In [25]:
# Sélectionner aléatoirement 3 hrefs
hrefs = random.sample(hrefs, 3)

In [26]:
hrefs

['https://professeur-layton.fandom.com/fr/wiki/Croix_de_bois...',
 'https://professeur-layton.fandom.com/fr/wiki/Contre-la-montre',
 'https://professeur-layton.fandom.com/fr/wiki/%C3%80_la_belle_%C3%A9toile']

On va ensuite créer une fonction qui pour chaque lien récupère les données qui nous intéressent : 

In [27]:
from selenium.common.exceptions import NoSuchElementException

def collecte_enigme(href):
    url = href
    
    # Configuration du navigateur (Chrome dans cet exemple)
    options = Options()
    options.add_argument("--headless")  # Exécuter Chrome en mode headless
    chrome_options.add_argument('--disable-search-engine-choice-screen') 
    chrome_options.add_argument('--disable-infobars')
    driver = webdriver.Chrome(options=options)
    
    # Ouvrir la page web
    driver.get(url)
    time.sleep(3)
    try:
        cookie = driver.find_element(By.XPATH ,'/html/body/div[7]/div/div/div[2]/div[2]/div[1]')
        cookie.click()
    except :
        pass
    try : 
        # Récupération du titre :
        title = driver.find_element(By.ID, 'firstHeading')
        title = title.text
    except : 
        title = None
    try:
        time.sleep(4)
        num_enigme = driver.find_element(By.XPATH ,'//*[@id="mw-content-text"]/div/div[1]/div[2]/table/tbody/tr[4]/td')
        num_enigme = num_enigme.text
       
    except :
        num_enigme = None
    try:
        
        # Récupération de l'image
        image_element = driver.find_element(By.CSS_SELECTOR, 'div.floatnone img')
        image = image_element.get_attribute("src")
    
    except:
        image=None
    
    # Récupération de l'énoncé de l'énigme :
    enigme_enonce = None
    try : 
        enonce = driver.find_element(By.ID, "Énoncé")
        enigme_enonce = enonce.find_elements(By.XPATH, "//span[@class='mw-headline' and @id='Énoncé']/ancestor::h2/following-sibling::p[following-sibling::h2]")
        enigme_enonce = "".join([elem.text for elem in enigme_enonce])
    except NoSuchElementException:
        enigme_enonce = image
        
    resolution_title = None
    
    try:
        resolution_title = driver.find_element(By.ID, "Résolution")
        resolution = resolution_title.find_elements(By.XPATH, "//span[@class='mw-headline' and @id='Résolution']/ancestor::h3/following-sibling::p[following-sibling::h3]")
        resolution = "\n".join([elem.text for elem in resolution])
    except Exception:
        try:
            solution = driver.find_element(By.ID, "Solution")
            # Chercher le premier 'div' frère de 'Solution' qui contient le diaporama
            slideshow_div = solution.find_element(By.XPATH, '//*[@id="slideshow-0"]/div/div[1]')

            # Localiser l'image spécifique à l'intérieur de ce diaporama
            image_element = slideshow_div.find_element(By.XPATH, './/img[@class="thumbimage"]')
            image_url = image_element.get_attribute('src')
            resolution = image_url
            
        except : 
            text_elements = solution.find_elements(By.XPATH, 'following::p')
            resolution = "\n".join([elem.text for elem in text_elements])
 
    # Création du DataFrame
    return pd.DataFrame([{'title': title, 'num_enigme':num_enigme, 'url': url, 'image': image, 'enigme': enigme_enonce, 'solution': resolution}])


On utilise ensuite cette fonction pour créer notre DataFrame :

In [28]:
df_final = pd.DataFrame()
for href in hrefs:
    print("Scraping de la page", href, "en cours...")
    df_final = pd.concat([df_final, collecte_enigme(href)], ignore_index=True)
print("Fin du Scraping")

Scraping de la page https://professeur-layton.fandom.com/fr/wiki/Croix_de_bois... en cours...
Scraping de la page https://professeur-layton.fandom.com/fr/wiki/Contre-la-montre en cours...
Scraping de la page https://professeur-layton.fandom.com/fr/wiki/%C3%80_la_belle_%C3%A9toile en cours...
Fin du Scraping


In [29]:
pd.set_option('display.max_row', 5)
pd.set_option('display.max_column', 6)
df_final

,title,num_enigme,url,image,enigme,solution
0,Croix de bois...,061,https://professeur-layton.fandom.com/fr/wiki/C...,https://static.wikia.nocookie.net/layton/image...,On a relié les épingles de ce pêle-mêle de man...,Beau travail !\nCette énigme est très facile l...
1,Contre-la-montre,005,https://professeur-layton.fandom.com/fr/wiki/C...,https://static.wikia.nocookie.net/layton/image...,Une horloge à aiguilles standard est composée ...,Bonne réponse !\nL'aiguille des minutes dépass...
2,À la belle étoile,046,https://professeur-layton.fandom.com/fr/wiki/%...,https://static.wikia.nocookie.net/layton/image...,Regardez ce magnifique ciel étoilé.Combien de ...,Bonne réponse !\nIl y a douze triangles en tou...


In [30]:
#Permet de mettre fin au driver
driver.quit()

## Activité 3 BONUS : Généralisation à plusieurs énigmes avec BeautifulSoup

In [31]:
from bs4 import BeautifulSoup, Tag
from typing import List
import requests
import random
import pandas as pd

#### Lancement d'une requête permettant de récupérer tout le code source de l'url à scraper

In [36]:
url = "https://professeur-layton.fandom.com/fr/wiki/Cat%C3%A9gorie:%C3%89nigmes"
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

#### Afficher le code source de la page

In [38]:
#print(soup.prettify())

#### Récuperer tous les liens des énigmes et n'en prendre que 5 aléatoirement

In [39]:
# Trouver tous les éléments <a> avec la classe "category-page__member-link"
links = soup.find_all('a', class_='category-page__member-link')

# Extraire les attributs href
hrefs = [link.get('href') for link in links]

# Afficher les hrefs
#print(hrefs)
#print(len(hrefs))

# Sélectionner aléatoirement 5 hrefs
hrefs = random.sample(hrefs, 5)

# Afficher les hrefs sélectionnés aléatoirement
#print(random_hrefs)
#print(len(random_hrefs))

In [40]:
hrefs

['/fr/wiki/Mise_en_plis',
 '/fr/wiki/Les_aiguilles_du_temps',
 '/fr/wiki/Embouteillages',
 '/fr/wiki/In%C3%A9quations_%3F',
 '/fr/wiki/Groom_service']

#### Créons une fonction permettant de répéter les étapes d'extraction de données

In [41]:
def collecte_enigme(racine, href):
    url = racine + href
    soup = BeautifulSoup(requests.get(url).text, "html.parser")

    # Extraction des métadonnées
    title = soup.find('meta', property="og:title").get("content")
    url_enigme = soup.find('meta', property="og:url").get("content")
    
    # Récupération de l'image (src le plus long)
    src_urls = [tag['src'] for tag in soup.find_all(src=True)]
    image = max(src_urls, key=len, default=None)
    
    # Récupération de l'énigme
    enigme = None
    try:
        start_element = soup.find('span', id='Énoncé').find_parent('h2')
        enigme = "\n".join(
            sibling.get_text() for sibling in start_element.find_next_siblings() 
            if sibling.name in ['p', 'ul'] and sibling.name != 'h2'
        )
    except:
        enigme = image
    
    # Récupération de la réponse
    try:
        reponse = soup.find('span', class_='mw-headline', id='Solution').find_next('p').get_text(strip=True)
    except:
        reponse = soup.find('a', class_='image')
        reponse = reponse.get('href') if reponse else "La réponse n'a pas été loadée correctement"
    
    # Récupération des indices
    indices = [
        content.get_text(strip=True)
        for tab, content in zip(soup.select('ul.wds-tabs li.wds-tabs__tab a'), soup.select('div.wds-tab__content'))
        if content.select_one('div[style*="overflow-y:auto"]')
    ]

    # Création du DataFrame
    return pd.DataFrame([{'title': title, 'url': url_enigme, 'image': image, 'enigme': enigme, 'indices': indices, 'solution': reponse}])


#### Utilisation de la fonction et stockage dans un DataFrame

In [42]:
racine = "https://professeur-layton.fandom.com"
df_final = pd.DataFrame()
for href in hrefs:
    df_final = pd.concat([df_final, collecte_enigme(racine,href)], ignore_index=True)

#### Visualisation

In [43]:
pd.set_option('display.max_row', 7)
pd.set_option('display.max_column', 6)
df_final

,title,url,image,enigme,indices,solution
0,Mise en plis,https://professeur-layton.fandom.com/fr/wiki/M...,https://static.wikia.nocookie.net/layton/image...,Une ville ne compte que deux coiffeurs. Il n'y...,[Vous ne vous coupez pas les cheveux vous-même...,La réponse est le coiffeur A.
1,Les aiguilles du temps,https://professeur-layton.fandom.com/fr/wiki/L...,https://static.wikia.nocookie.net/layton/image...,L'horloge indique actuellement 15 h 30. Cela s...,[Il n'est pas nécessaire de toucher les aiguil...,"La réponse est 00, car il faut juste attendre ..."
2,Embouteillages,https://professeur-layton.fandom.com/fr/wiki/E...,https://static.wikia.nocookie.net/layton/image...,"La Laytonmobile, le bijou du professeur, est c...",[Cela ne sert à rien d'essayer de libérer la v...,Beau travail !
3,Inéquations ?,https://professeur-layton.fandom.com/fr/wiki/I...,https://static.wikia.nocookie.net/layton/image...,"Eh bien, on dirait que quelqu'un a encore écri...","[Au premier coup d'œil, il semble que l'auteur...",La réponse est 1.
4,Groom service,https://professeur-layton.fandom.com/fr/wiki/G...,https://static.wikia.nocookie.net/layton/image...,Deux portiers doivent prendre six valises dont...,"[Puisqu'il y a six valises, il est permis de s...",Beau Travail!


# Conclusion

Nous avons finalement réussi à se constituer un DataFrame à partir d'un site web ! 
Ce qu'il faut retenir, c'est que BeautifulSoup est très utile pour les sites dit "ouverts" et pour de la répetition massive de collecte d'infos.